In [1]:
from nbodykit.lab import *
from nbodykit import style, setup_logging

import corner
from tqdm import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

import sys, pickle, time, os, h5py

from nbodykit.source.catalog import BinaryCatalog
# nbodykit cosmology parameters initialization
from nbodykit.lab import FFTPower

In [2]:
COSMOPAR = {
#                   | Om   | Ob   |   h   |  n_s  | s_8 | Mnu | w |

    'fiducial' :    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'zeldovich':    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Mnu_p' :       [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.1, -1],
    'Mnu_pp' :      [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.2, -1],
    'Mnu_ppp' :     [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.4, -1],
    
    'h_m' :         [0.3175, 0.049, 0.6511, 0.9624, 0.834, 0, -1],
    'h_p' :         [0.3175, 0.049, 0.6911, 0.9624, 0.834, 0, -1],
    
    'ns_m' :        [0.3175, 0.049, 0.6711, 0.9424, 0.834, 0, -1],
    'ns_p' :        [0.3175, 0.049, 0.6711, 0.9824, 0.834, 0, -1],
    
    'Ob_m' :        [0.3175, 0.048, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob_p' :        [0.3175, 0.050, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_m' :       [0.3175, 0.047, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_p' :       [0.3175, 0.051, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Om_m' :        [0.3075, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'Om_p' :        [0.3275, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    's8_m' :        [0.3175, 0.049, 0.6711, 0.9624, 0.819, 0, -1],
    's8_p' :        [0.3175, 0.049, 0.6711, 0.9624, 0.849, 0, -1]
}

folders_ordered = {
    0 :'fiducial'  ,    
    1 :'h_m'       , 
    2 :'h_p'       ,    
    3 :'Mnu_p'     , 
    4 :'Mnu_pp'    ,
    5 :'Mnu_ppp'   ,
    6 :'ns_m'      , 
    7 :'ns_p'      ,    
    8 :'Ob2_m'     , 
    9 :'Ob2_p'     ,    
    10 :'Om_m'      , 
    11 :'Om_p'      ,
    12 :'s8_m'      , 
    13 :'s8_p'      ,
    14 :'zeldovich' ,}

order_folders = {
    'fiducial'  : 0,
    'h_m'       : 1,  'h_p'       : 2,
    'Mnu_p'     : 3,  'Mnu_pp'    : 4, 'Mnu_ppp'   : 5, 
    'ns_m'      : 6,  'ns_p'      : 7,
    'Ob2_m'     : 8,  'Ob2_p'     : 9,
    'Om_m'      : 10, 'Om_p'      : 11,
    's8_m'      : 12, 's8_p'      : 13,
    'zeldovich' : 14
}

order_dimension = {
    'Om'  : 0, 'Om ' : 0,
    'Ob'  : 1, 'Ob ' : 1, 'Ob2' : 1,
    'h'   : 2, 'h  ' : 2, # 4
    'ns'  : 3, 'ns ' : 3,
    's8'  : 4, 's8 ' : 4, # 2
    'Mnu' : 5
}

"""Om, Ob, s8, ns, h, Mnu"""

cosmological_pars = {
    'Om'  : 0, 'Ob'  : 1, 'h'   : 2,
    'ns'  : 3, 's8'  : 4, 'Mnu' : 5
}

VarCosmoPar = {
    'd_h'  : 0.02, 'd_ns' : 0.02,  'd_Ob' : 0.001, 'd_Ob2': 0.002,
    'd_Om' : 0.01, 'd_s8' : 0.015
}

fiducial_vals = {
    'Ob'  : 0.3175, 'Ob2' : 0.3175,
    'Om'  : 0.049,
    'h'   : 0.6711,
    'n_s' : 0.9624, 'ns'  : 0.9624,
    's_8' : 0.834,  's8'  : 0.834,
    'Mnu' : 0
}

In [3]:
def correlation_matrix(m):
    """Calculates the correlation matrix of matrix m_ij."""
    avg = np.average(m, axis=0)
    dim = len(avg)
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # VARIATION
    sigma = []
    for i in range(dim):
        sigma.append(np.sum((avg[i] - m[i])**2))
    sigma = np.sqrt(sigma)
    
    sigma_bis = []
    for i in range(dim):
        cum = 0
        for j in range(len(m[i])):
            cum += (avg[i] - m[i][j])**2
        sigma_bis.append(cum)
    sigma_bis = np.sqrt(sigma_bis)        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # CORRELATION MATRIX
    CORR = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            cum = 0
            for k in range(len(m[i])):
                cum += (avg[i] - m[i][k])*(avg[j] - m[j][k])
            CORR[i, j] = cum  / (sigma_bis[i] * sigma_bis[j])

    if np.linalg.det(CORR) == 0:
        print("¶ WARNING: correlation matrix is singular")
    
    return CORR

def Hartlap(mat, Nr = 350):
    """Calculates inverse matrix using Hartlap correction.
    Arguments:
    - `mat`: input matrix to invert
    - `Nr`: nuber of realization used o calculated the matrix
    """
    return (Nr-len(mat)-2)/(Nr-1)*np.linalg.inv(mat)

def info_name(name):
    """Obtain realization information from namefile"""
    print("TYPE: ", type(name))
    info = name.split('_')[-3:]
    print("TYPE: ", info, "\n", type(info))
    info[2] = info[2].replace(".wst", "")
    N_hgrid = info[0]
    N_WSTgrid = info[1]
    n_realiz = info[2].replace(".wst", "")
    
    return [int(N_hgrid), int(N_WSTgrid), int(n_realiz)]

def cosmo_parser(name):
    """Obtain cosmology from .wst file"""
    info = name.split('_')
    if info[0] == "fiducial":
        return info[0]
    elif info[0] == "zeldovich":
        return info[0]
    else:
        return info[0] + "_" + info[1]
    
def covariation_matrix(m):
    """Calculates the correlation matrix of matrix m_ij.
    """
    print('Shape: ' + str(np.shape(m)))
    avg = np.average(m, axis=0)    # mean values
    dim = len(avg)                 # number of variables
    N = len(m[0])                  # number of values per varaible
    print('len matr ' + str(len(m[0])))
    print('   len avg ' + str(dim))
    COV = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            cum = 0.
            for k in range(len(m[i])):
                cum += (avg[i] -
                         m[i][k])*\
                    (avg[j] - 
                     m[j][k])
            COV[i, j] = cum/N

    if np.linalg.det(COV) == 0:
        print("¶ WARNING: correlation matrix is singular")
    
    return COV

def error_message(a = "ERROR >.<"):
    N = len(a)
    print("")
    print(" "*6 + "_"*N)
    print(" "*5+"/" + " "*N + "\\")
    print(" === | " + a + "| ===")
    print(" "*5 + "\\" + "_"*N + "/" )
    print("")

def mean_coeffs(M):
    X, Y = np.shape(M)
    r = np.zeros(Y)
    M *= 1e-14
    for j in range(Y):  # indice coeff
        c = 0
        for i in range(X): # indice realiz
            c += (M[i][j]/X)
        r[j] = c
    return r*1e14

In [4]:
root_WST = "/home/fuffolo97/TESI/WST_3D_files/"
root_Pk  = "/home/fuffolo97/TESI/Pk_3D_files/"
files_to_read_wst = os.listdir(root_WST)
files_to_read_pk = os.listdir(root_Pk)


In [5]:
folders = [
    'fiducial', 'zeldovich', 'h_m', 'h_p', 'Mnu_p', 'Mnu_pp' ,'Mnu_ppp',
    'ns_m', 'ns_p', 'Ob2_m', 'Ob2_p',
    'Om_m', 'Om_p', 's8_m', 's8_p'
]
cosmologies = [
    'fiducial', 'zeldovich', 'h_m', 'h_p', 'Mnu_p', 'Mnu_pp' ,'Mnu_ppp',
    'ns_m', 'ns_p', 'Ob2_m', 'Ob2_p',
    'Om_m', 'Om_p', 's8_m', 's8_p'
]
parameter_name = ['Om ', 'Ob ', 'h  ', 'ns ', 's8 ', 'Mnu']

In [6]:
fiducial_wst_real = []
zeldovich_wst_real = []
fiducial_pk_real = []
zeldovich_pk_real = []
coeffs_wst = np.zeros((len(cosmologies), 75))
coeffs_pk  = np.zeros((len(cosmologies), 159))

In [7]:
files_to_read_pk.remove('ArrayK.hdf5')

In [8]:
every_mean_wst = np.zeros((len(order_folders), 75))

for FC in range(len(files_to_read_wst)):
    every_wst = []
    fname = files_to_read_wst[FC]
    cosmo = cosmo_parser(files_to_read_wst[FC])
    assert cosmo in files_to_read_wst[FC]
    index = order_folders[cosmo]

    if fname == 'fiducial': num = 1000
    elif 'ZA' in fname: num = 500
    else: num = 350

    with h5py.File(root_WST+fname, 'r') as f:
        for i in range(num):
            dat = np.array(f[f'{cosmo}_{i}_wst'])
            every_wst.append(dat)
    every_mean_wst[index] = np.mean(every_wst, axis=0)

In [9]:
every_mean_pk = np.zeros((len(order_folders), 159))
real_power = np.zeros((len(order_folders), 159))
every_mean_Nnoise = np.zeros((len(order_folders), 159))
all_noises = []

for FC in tqdm(range(len(files_to_read_pk))):
    every_pk = []
    every_pk_Nnoise = []
    every_power = []
    fname = files_to_read_pk[FC]
    cosmo = cosmo_parser(files_to_read_pk[FC])
    assert cosmo in files_to_read_pk[FC]
    index = order_folders[cosmo]

    if fname == 'fiducial': num = 1000
    elif 'ZA' in fname: num = 500
    else: num = 350

    with h5py.File(root_Pk+fname, 'r') as f:
        for i in range(num):
            pk_real = np.array(f[f'{cosmo}_{i}_pk'])
            noise = np.array(f[f'{cosmo}_{i}_noise'])
            every_pk.append(pk_real)
            every_pk_Nnoise.append(pk_real-noise)
    every_mean_pk[index] = np.mean(every_pk, axis=0)
    every_mean_Nnoise[index] = np.mean(every_pk_Nnoise, axis=0)

100%|██████████| 15/15 [00:02<00:00,  6.71it/s]


In [10]:
deriavates_wst = np.zeros((len(cosmological_pars),  75))
deriavates_pk  = np.zeros((len(cosmological_pars), 159))

for i in cosmological_pars:
    if "Mnu" not in i and "Ob" not in i:
        ind = order_dimension[i]
        deriavates_wst[ind]=(every_mean_wst[order_folders[i+"_p"]]-every_mean_wst[order_folders[i+"_m"]])\
                    /  (2 * VarCosmoPar['d_'+i] * fiducial_vals[i] )
        deriavates_pk[ind]=(real_power[order_folders[i+"_p"]]-real_power[order_folders[i+"_m"]])\
                    /  (2 * VarCosmoPar['d_'+i] * fiducial_vals[i] )

    elif "Mnu" in i:
        deriavates_wst[order_dimension['Mnu']] = (every_mean_wst[order_folders["Mnu_p"]] - every_mean_wst[order_folders["zeldovich"]]) / (0.1)
        deriavates_pk[order_dimension['Mnu']]  = (real_power[order_folders["Mnu_p"]]  - real_power[order_folders["zeldovich"]])  / (0.1)

    elif "Ob" in i:
        deriavates_wst[order_dimension['Ob']] = \
            (every_mean_wst[order_folders[i+"2_p"]]-\
             every_mean_wst[order_folders[i+"2_m"]]) \
                / (2 * VarCosmoPar['d_'+i+"2"] * fiducial_vals[i] )
        deriavates_pk[order_dimension['Ob']] = \
            (real_power[order_folders[i+"2_p"]]-\
             real_power[order_folders[i+"2_m"]]) \
                / (2 * VarCosmoPar['d_'+i+"2"] * fiducial_vals[i] )


In [12]:
# create transposed array of Pk coefficients
fiducial_wst = every_mean_wst[order_folders['fiducial']].transpose()
fiducial_pk = every_mean_pk[order_folders['fiducial']].transpose()

In [13]:
fiducial_wst = []
fiducial_pk = []

with open("./WST_3D_files/fiducial_coefficients_real.wst", "rb") as f:
    while True:
        try:
            fiducial_wst.append(pickle.load(f))
        except EOFError:
            break

with open("./Pk_3D_files/fiducial_coefficients_real.pk", "rb") as f:
    while True:
        try:
            a = pickle.load(f)
            a = a['power'].real
            fiducial_pk.append(a)
        except EOFError:
            break

# create transposed array of Pk coefficients
fiducial_wst = np.array(fiducial_wst).transpose()
fiducial_pk = np.array(fiducial_pk).transpose()

# cov_wst = covariation_matrix(fiducial_wst)
# cov_pk  = covariation_matrix(fiducial_pk)

cov_wst = np.cov(fiducial_wst)
cov_pk  = np.cov(fiducial_pk)

# inverting with Hartlap factor
Hart_wst = Hartlap(cov_wst, 1000)
Hart_pk = Hartlap(cov_pk, 1000)

# initialize empty Fisher matrix
Fish_wst, Fish_pk  = np.zeros((6,6)), np.zeros((6,6))

# create Fisher matrix
for a in range(6):
    for b in range(6):
        Fish_wst[a, b]  = np.sum(deriavates_wst[a]  * (Hart_wst  * deriavates_wst[b]))
        Fish_pk[a, b]   = np.sum(deriavates_pk[a]   * (Hart_pk   * deriavates_pk[b]))

# GETTING CONATRAINS:
# create inverse of Fisher matrix
inverse_wst = np.linalg.inv(Fish_wst)
inverse_pk = np.linalg.inv(Fish_pk)

# initialize empty array containing diagonal elements
diagonal_wst, diagonal_pk = [], []

# inizialize empty matrix containing all correletions
constrains_wst, constrains_pk = np.zeros((6,6)), np.zeros((6,6))

# `for loop`: assegnate diaconal elements
for i in range(len(inverse_wst)):
    diagonal_wst.append(inverse_wst[i, i])
    diagonal_pk.append(inverse_pk[i, i])
    # internal `for loop`: assegnate all the correlations
    for j in range(6):
        constrains_wst[i, j] += inverse_wst[i, j]
        constrains_pk[i, j] += inverse_pk[i, j]

# after checking symmetric elements are almost equal, set them as equal
# this is needed because exact symm. matrix is needed
for i in range(6):
    for j in range(6):
        assert (np.abs(constrains_wst[i, j] - constrains_wst[j, i]) < 1e-5), f'{np.abs(constrains_wst[i, j] - constrains_wst[j, i])}'
        assert (np.abs(constrains_pk[i, j] - constrains_pk[j, i]) < 1e-5), f'{np.abs(constrains_pk[i, j] - constrains_pk[j, i])}'
        constrains_wst[i, j] = np.abs(constrains_wst[j, i])
        constrains_pk[i, j] = constrains_pk[j, i]

# # create correlation graphs, maybe useful to confront with wst
# sns.heatmap(corr.transpose())
# plt.gca().invert_yaxis()
# plt.savefig('correlation_matrix_pk.pdf', format='pdf')

# sns.heatmap(corr_rsd.transpose())
# plt.gca().invert_yaxis()
# plt.savefig('correlation_matrix_rsd_pk.pdf', format='pdf')

# # save constrains in a file
# with open("./ZZ_results/constrains_pk.res", "ab") as f:
#     pickle.dump(constrains, f)
# with open("./ZZ_results/constrains_rsd_pk.res", "ab") as f:
#     pickle.dump(constrains_rsd, f)

LinAlgError: Singular matrix